In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import time 
import datetime
import shutil

%matplotlib inline

class RUN:
    timePerOrbit = 110*24.95
            
    def __init__(self, dataFrame):
        
        
        self.df = dataFrame
        self.events = len(self.df)

        
        self.orbitMin  = self.df['orbit'][:1].values[0]
        self.orbitMax  = self.df['orbit'][-1:].values[0] 
        self.nOrbits= self.orbitMax - self.orbitMin
        self.rectime   = self.nOrbits * RUN.timePerOrbit # in ns
        self.rectimeSECONDS   = self.nOrbits * RUN.timePerOrbit * 10**(-9) # in s
        
        if (self.df['run'][0] == self.df['run'][-1:].values[0]): # This code is not intended for use on merged runs
            self.run = self.df['run'][0]   
            TimeLogPath = "https://raw.githubusercontent.com/theofil/dscout/master/analysis/runsDateTimeLog.txt"
            TimeLog = pd.read_csv(TimeLogPath, delimiter = ', ', engine = 'python')

            for i in range(len(TimeLog['run'])):
                if TimeLog['run'][i] == self.run : 
                    index = i 
                    break

            duration = TimeLog['duration'][index]
            self.tStart = TimeLog['time_start'][index]
            self.tEnd = TimeLog['time_end'][index]
            h, m, s=duration.split(":")
            durationSecs = datetime.timedelta(hours = int(h), minutes = int(m), seconds = int(s)).total_seconds()
    #       --------------------------        
            self.duration = durationSecs
            self.naiveRate = self.events/durationSecs # in seconds
    #       --------------------------    
    
    def rate(self, timeUnit = 1, graph = False, info = False):
        
        orbitMin = self.orbitMin 
        orbitMax = self.orbitMax
        df = self.df['orbit']
        timeUnit = timeUnit*1e9 # desired time in seconds
        orbitsPerTimeUnit = timeUnit/RUN.timePerOrbit
        bins=int(self.nOrbits/orbitsPerTimeUnit) 
#         Depending on the time unit, some runs might have   orbitsPerTimeUnit > self.nOrbits
        if bins == 0: bins = 1  
        timePerBin = self.rectime/bins
        muons=np.zeros(bins)
        sclice=np.array([orbitMin+ ((orbitMax-orbitMin)/bins)*i for i in range(bins+1)])

        # Count the events in the first orbit = orbitMin
        for orbit in df:
            if(orbit > orbitMin): break
            if(orbit == orbitMin): muons[0] += 1

        df = df[int(muons[0]):] # We already counted the first orbit
        i = 0
        for orbit in df:
            while not (orbit > sclice[i] and orbit <= sclice[i + 1]) and i < bins-1: # Note the <= in 'orbit <= slicePlusOne[i]'. 
                i = i + 1                                                              # It is needed to capture the last orbit.
            muons[i] += 1

        # Maximum Likelihood Estimation of Poisson parameter
        Nmuons = int(muons.sum()) 
        Ninterval = len(muons) # How many time intervals

        mu = muons.mean()
        med = np.median(muons)
        sigma = muons.std()
        MLE = Nmuons/Ninterval

        # 99% Confidence interval under gaussian approximation

        error = 2.58 * np.sqrt(MLE/Ninterval)
        lower = MLE-error
        upper = MLE+error
        
        if (info):
        
            print('timePerOrbit %2.1f ns '%RUN.timePerOrbit)
            print('orbitMin %d orbitMax %d'%(self.orbitMin, self.orbitMax))
            print('nOrbits %d '%self.nOrbits)
            print('total recording time %2.4f ns (~= %2.1f s)' %(self.rectime, self.rectime*1.e-9))
            print('orbitsPerTimeUnit = %2.3f'%orbitsPerTimeUnit)
            print("bins = %d" %bins)
            print('--------------------------------')
            print('mean = {:.2f}' .format(mu))
            print('median = {:d}'.format(int(med)))             
            print('sample std = {:.2f}'.format( sigma))
            #print('Gaussian 99% Confidence Interval = [{:.2f} , {:.2f} ]'.format(lower, upper))
            
        if (graph):
            
            if style == 1:
                
                muons = pd.Series(muons)
                counts = muons.value_counts().sort_index()
                
                plt.figure(figsize=(8,8))
                plt.errorbar(counts.index, counts.values, xerr = None, yerr = np.sqrt(counts.values), fmt = 'o')
                plt.xlabel('Muons / %2.1f s'%(timePerBin*1.e-9), fontsize = 20) # latex can be entered in the label's string
                plt.ylabel('Frequency', fontsize = 20)

                plt.show()
                
            if style == 2:
            
                muons = np.array(muons)
                counts, binEdges = numpy.histogram(muons, bins=len(muons))

                bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
                menStd     = np.sqrt(counts)
                width      = 1

                plt.figure(figsize=(8,8))
                plt.bar(bincenters, counts, width=width, color='b', ecolor = 'r', yerr=menStd, error_kw = {'alpha': 1})
                plt.xlabel('Muons / %2.1f s'%(timePerBin*1.e-9), fontsize = 20) # latex can be entered in the label's string
                plt.ylabel('Frequency', fontsize = 20)

                plt.show()

        return mu  # muons.mean() is the rate (events per timeUnit)


    def plotPhi(self, bins = 30):
        
        phiticks = [0.5*np.pi*i for i in range(-2,3)]
        plt.rc('font', size=22)
        plt.figure(figsize=(8,8))
        plt.hist(self.df['phi'], bins = bins, ec='black', histtype = 'stepfilled')
        plt.xticks(phiticks)
        plt.xlabel('$\phi$ [radians]')
        plt.ylabel('Muons')
        plt.title('Run [{}], Duration = {:.2f} min '.format(self.run, self.duration/60), size = 30)
        plt.show()
        
    def plotOrbit(self, bins = 100):
        
        plt.rc('font', size=22)
        plt.figure(figsize=(8,8))
        plt.hist(self.df['orbit'], bins = bins, ec='black', histtype = 'stepfilled')
        plt.xlabel('Orbit', size = 30)
        plt.ylabel('Muons', size = 30)
        plt.xticks(size = 20)
        plt.yticks(size = 20)
        plt.title('Run [{}], Duration = {:.2f} min '.format(self.run, self.duration/60), size = 30)
        plt.show()
        
        
def pair(df, x):

    dfns = df.shift(-x)
    dfps = df.shift(+x)

    logicOS1 = (df.bx == dfns.bx - x) & (df.orbit == dfns.orbit) & (df.charge*dfns.charge < 0)
    OS1 = df[logicOS1].copy()

    logicOS2 = (df.bx == dfps.bx + x) & (df.orbit == dfps.orbit) & (df.charge*dfps.charge < 0)
    OS2 = df[logicOS2].copy()

    OS1 = OS1.reset_index()
    OS2 = OS2.reset_index()

    OS = OS1.merge(OS2, left_on=OS1.index, right_on=OS2.index, suffixes=('_in', '_out'))

    logicSS1 = (df.bx == dfns.bx - x) & (df.orbit == dfns.orbit) & (df.charge*dfns.charge > 0)
    SS1 = df[logicSS1].copy()

    logicSS2 = (df.bx == dfps.bx + x) & (df.orbit == dfps.orbit) & (df.charge*dfps.charge > 0)
    SS2 = df[logicSS2].copy()

    SS1 = SS1.reset_index()
    SS2 = SS2.reset_index()

    SS = SS1.merge(SS2, left_on=SS1.index, right_on=SS2.index, suffixes=('_in', '_out'))

    # slim the dataframes
    def slimDataFrame(df):
        df = df.drop(['run_in', 'index_in','index_out','key_0','orbit_out'], axis=1)
        df = df.rename(columns = {'run_out':'run','orbit_in':'orbit'})
        if False: df = df.set_index(['run','orbit','bx_in'])
        return df

    OS = slimDataFrame(OS)
    SS = slimDataFrame(SS)
    return (OS, SS)

In [5]:

if not os.path.exists('./data/Merged/'):
        os.makedirs('./data/Merged/')



# data files
path = './data/goodRuns/csv/'
files = os.listdir(path)
files = [f for f in files]

# getrunNumber from string e.g., int('/data/hiion/scout_326676_000000.monitor.txt'.split('_')[1]
getRun = lambda x: int(x.split('.')[0])



# list to hold dataframes
dfList = []
runIndex= []

for file in files:
    filepath = path+file
    df = pd.read_csv(filepath)
    
  
    # add a column with the run number
    df['run'] = getRun(file) 
    dfList += [df]
    
    #make in index with the run numbers
    runIndex += [getRun(file)]


In [6]:
#Fix the runs where the orbit number resets (regard only runs after the reset)

dfs = []
for df in dfList:
    reset = 0
    if not df['orbit'].is_monotonic:
        for i in range(1,len(df)):
            if df.loc[i-1, 'orbit']>df.loc[i, 'orbit']:
                reset = i
                break
    tmp = df.loc[reset:]
    tmp.reset_index(drop=True, inplace=True) 
    dfs.append(tmp)            

In [7]:
badrunNo = [326262, 326303, 326587, 327378, 327424, 327430, 327455, 327488, 327489]

In [9]:
dfg = [] #data frame with only goodruns

for df in dfs:
    if df['run'][0] not in badrunNo : dfg.append(df)

In [10]:
# Make every df start at 1
for df in dfg:
    df.loc[:, 'orbit'] = df.loc[:, 'orbit'] - df.loc[0, 'orbit'] + 1

In [11]:
for j in range(1, len(dfg)):
    dfg[j].loc[:, 'orbit']+=dfg[j-1].loc[:, 'orbit'].tolist()[-1]

In [12]:
merged = pd.concat(dfg, ignore_index=True, sort=False)


In [13]:
merged.to_csv(path_or_buf="./data/Merged/merged.txt", index=False ) 
# # so that someone else can intstant access to the data without running this code

# Two - Leg Muon filtering

# ΔBx = 1

In [15]:
OS1, SS1 = pair(merged, 1)


In [16]:
len(OS1)/len(merged)

0.08071439132916572

In [17]:
OS1.head()

,orbit,bx_in,phi_in,eta_in,pt_in,charge_in,bx_out,phi_out,eta_out,pt_out,charge_out,run
0,41208,3458,2.39983,0.000000,4.0,-1,3459,-1.15628,0.80475,3.5,1,326305
1,86938,3481,1.09083,-0.511125,16.5,1,3482,-1.45081,-0.76125,17.5,-1,326305
2,98659,3511,1.12356,0.511125,2.5,1,3512,-2.00713,-0.45675,38.0,-1,326305
3,106772,3458,1.84350,0.511125,139.5,1,3459,-1.02538,0.80475,139.5,-1,326305
4,122121,3476,2.17075,0.804750,3.5,1,3477,-1.61443,0.80475,3.5,-1,326305


In [18]:
OS1.to_csv(path_or_buf="./data/Merged/twolegOS1.txt", index=False ) 
# so that someone else can intstant access to the data without running this code

In [19]:
SS1.to_csv(path_or_buf="./data/Merged/twolegSS1.txt", index=False ) 
# so that someone else can intstant access to the data without running this code

# ΔBx = 2

In [20]:
OS2, SS2 = pair(merged, 2)

In [21]:
len(OS2)/len(merged)

0.02442325678956309

In [22]:
OS2.head()

,orbit,bx_in,phi_in,eta_in,pt_in,charge_in,bx_out,phi_out,eta_out,pt_out,charge_out,run
0,41208,3457,2.39983,0.000000,3.5,-1,3459,-1.156280,0.804750,3.5,1,326305
1,86938,3480,1.07992,-0.358875,26.5,1,3482,-1.450810,-0.761250,17.5,-1,326305
2,98659,3510,1.11265,0.511125,30.0,1,3512,-2.007130,-0.456750,38.0,-1,326305
3,169057,3514,1.65806,0.000000,4.5,-1,3516,-0.970839,-0.641625,5.5,1,326305
4,212226,3540,1.69079,-0.358875,4.5,-1,3542,-0.807215,-0.467625,4.5,1,326305


In [23]:
OS2.to_csv(path_or_buf="./data/Merged/twolegOS2.txt", index=False ) 
# so that someone else can intstant access to the data without running this code

In [24]:
SS2.to_csv(path_or_buf="./data/Merged/twolegSS2.txt", index=False ) 
# so that someone else can intstant access to the data without running this code

# ΔBx = 3

In [25]:
OS3, SS3 = pair(merged, 3)

In [26]:
len(OS3)/len(merged)

0.001223720316078318

In [27]:
OS3.head()

,orbit,bx_in,phi_in,eta_in,pt_in,charge_in,bx_out,phi_out,eta_out,pt_out,charge_out,run
0,553338,3552,1.54898,0.000000,7.0,1,3555,-2.039850,-0.804750,5.0,-1,326305
1,4426285,3453,1.53807,0.000000,6.5,1,3456,-1.374450,-0.054375,8.0,-1,326305
2,5824744,3552,1.38535,0.511125,7.5,-1,3555,-1.407170,0.685125,5.5,1,326305
3,6817027,3552,1.23264,-0.467625,10.0,1,3555,-2.083490,0.000000,9.0,-1,326305
4,6859127,3551,1.63625,-0.358875,7.0,1,3554,-0.916298,0.000000,6.5,-1,326305


In [28]:
OS3.to_csv(path_or_buf="./data/Merged/twolegOS3.txt", index=False ) 
# so that someone else can intstant access to the data without running this code

In [29]:
SS3.to_csv(path_or_buf="./data/Merged/twolegSS3.txt", index=False ) 
# so that someone else can intstant access to the data without running this code